In [129]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchvision import datasets, transforms
from torch.utils.data import random_split
from matplotlib import pyplot as plt

torch.manual_seed(123)

In [130]:
# Based on code from the pytorch tutorials

data_path = '../data/'
bp_filter = lambda x: x[1] in [0,1] # Bird and Plane Filter. Or if you will, Everything but Bird and Plane Filter
preprocessor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4915, 0.4823, 0.4468),
                                (0.2470, 0.2435, 0.2616))
        ])

cifar10_train_val = list(filter(bp_filter, datasets.CIFAR10(data_path, train=True, download=True, transform=preprocessor)))
cifar10_test = list(filter(bp_filter, datasets.CIFAR10(data_path, train=False, download=True, transform=preprocessor)))

label_map = {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
class_names = ['not', 'airplane']

# For each dataset, keep only airplanes and birds
#cifar2_train_val = [(img, label_map[label]) for img, label in cifar10_train_val if label in [0, 2]]
#cifar2_test = [(img, label_map[label]) for img, label in cifar10_test if label in [0, 2]]

cifar2_train_val = [(img, label_map[label]) for img, label in cifar10_train_val]
cifar2_test = [(img, label_map[label]) for img, label in cifar10_test]


n_train = int(len(cifar2_train_val)*0.90)
n_val =  len(cifar2_train_val) - n_train
cifar2_train, cifar2_val = random_split(
        cifar2_train_val, 
        [n_train, n_val],
        generator=torch.Generator().manual_seed(123)
)


Files already downloaded and verified
Files already downloaded and verified


In [131]:
class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()  # to inherit the '__init__' method from the 'nn.Module' class
        # Add whatever you want here (e.g layers and activation functions)
        # The order and names don't matter here but it is easier to understand
        # if you go for Layer1, fun1, layer2, fun2, etc
        # Some conventions:
        # - conv stands for convolution
        # - pool for pooling
        # - fc for fully connected

        # 32*32*3: determined by our dataset: 32x32 RGB images
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 5, stride = 1, padding = 0)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5, stride = 1, padding = 0)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = 5, stride = 1, padding = 0)
        
        self.flat = nn.Flatten()
        
        self.fc1 = nn.Linear(in_features = 120 , out_features = 84) # 5 is height 8 is width and 120 is nr channels.
        self.fc2 = nn.Linear(in_features = 84 , out_features = 10)
        self.fc3 = nn.Linear(in_features = 10 , out_features = 2) #C + 5 components
         
        
             

    def forward(self, x):
        # Now the order matters! 
        out = self.act1(self.conv1(x))
        out = self.pool1(out)
        out = self.act1(self.conv2(out))
        out = self.pool2(out)
        out = self.act1(self.conv3(out))
        out = self.flat(out)
        out = self.act1(self.fc1(out))
        out = self.act1(self.fc2(out))
        out = self.fc3(out)
        
        return out
        

In [132]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    
    n_batch = len(train_loader)
    
    # We'll store there the training loss for each epoch
    losses_train = []
    
    # Set the network in training mode
    model.train()
    
    # Re-initialize gradients, just in case the model has been inappropriately 
    # manipulated before the training
    optimizer.zero_grad(set_to_none=True)
    
    for epoch in range(1, n_epochs + 1): 
        
        # Training loss for the current epoch
        loss_train = 0

        # Loop over our dataset (in batches the data loader creates for us)
        for imgs, labels in train_loader:
            
            # Feed a batch into our model
            outputs = model(imgs)
            
            # Compute the loss we wish to minimize 
            # Note that by default, it is the mean loss that is computed
            # (so entire_batch_loss / batch_size)
            loss = loss_fn(outputs, labels) 
            #print(loss)
            
            # Perform the backward step. That is, compute the gradients of all parameters we want the network to learn
            loss.backward()
            
            # Update the model
            optimizer.step() 
            
            # Zero out gradients before the next round (or the end of training)
            optimizer.zero_grad() 

            # Update loss for this epoch
            # It is important to transform the loss to a number with .item()
            loss_train += loss.item()
            
        # Store current epoch loss. 
        losses_train.append(loss_train / n_batch)
        

        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
            print(losses_train) 
            
    return losses_train

In [ ]:
def compute_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0

    # We do not want gradients here, as we will not want to update the parameters.
    with torch.no_grad():
        for imgs, labels in loader:

            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

    acc =  correct / total
    print("Accuracy: {:.2f}".format(acc))
    return acc

In [133]:
train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=200, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=200, shuffle=False)
test_loader = torch.utils.data.DataLoader(cifar2_test, batch_size=200, shuffle=False)

In [134]:
torch.manual_seed(123)
model_MyMLP = MyMLP()
loss_fn = nn.CrossEntropyLoss()


optimizer = optim.SGD(model_MyMLP.parameters(), lr=0.03)
# optimizer = optim.SGD(model_MyMLP.parameters(), lr=1e-2)
# loss_train = train(10, optimizer, model_1, loss_fn, train_loader)

train(20, optimizer, model_MyMLP, loss_fn, train_loader)

loss_train = train(20, optimizer, model_MyMLP, loss_fn, train_loader)
loss_train

09:41:02.244024  |  Epoch 1  |  Training loss 0.698
[0.6983958270814684]
09:41:48.541964  |  Epoch 10  |  Training loss 0.698
[0.6983958270814684, 0.6983958217832777, 0.69839582045873, 0.6983958257569207, 0.698395828406016, 0.698395832379659, 0.6983958138359918, 0.6983958151605394, 0.6983958151605394, 0.6983958085378011]
09:42:39.535235  |  Epoch 20  |  Training loss 0.698
[0.6983958270814684, 0.6983958217832777, 0.69839582045873, 0.6983958257569207, 0.698395828406016, 0.698395832379659, 0.6983958138359918, 0.6983958151605394, 0.6983958151605394, 0.6983958085378011, 0.6983958178096348, 0.6983958164850871, 0.69839582045873, 0.6983958151605394, 0.6983958231078253, 0.6983958429760403, 0.6983958270814684, 0.6983958125114441, 0.6983958098623487, 0.6983958164850871]


[0.6983958270814684,
 0.6983958217832777,
 0.69839582045873,
 0.6983958257569207,
 0.698395828406016,
 0.698395832379659,
 0.6983958138359918,
 0.6983958151605394,
 0.6983958151605394,
 0.6983958085378011,
 0.6983958178096348,
 0.6983958164850871,
 0.69839582045873,
 0.6983958151605394,
 0.6983958231078253,
 0.6983958429760403,
 0.6983958270814684,
 0.6983958125114441,
 0.6983958098623487,
 0.6983958164850871]

In [136]:
compute_accuracy(model_MyMLP, val_loader)

09:42:44.046047  |  Epoch 1  |  Training loss 0.698
[0.6983958270814684]
09:43:25.034933  |  Epoch 10  |  Training loss 0.698
[0.6983958270814684, 0.6983958217832777, 0.69839582045873, 0.6983958257569207, 0.698395828406016, 0.698395832379659, 0.6983958138359918, 0.6983958151605394, 0.6983958151605394, 0.6983958085378011]
09:44:16.725736  |  Epoch 20  |  Training loss 0.698
[0.6983958270814684, 0.6983958217832777, 0.69839582045873, 0.6983958257569207, 0.698395828406016, 0.698395832379659, 0.6983958138359918, 0.6983958151605394, 0.6983958151605394, 0.6983958085378011, 0.6983958178096348, 0.6983958164850871, 0.69839582045873, 0.6983958151605394, 0.6983958231078253, 0.6983958429760403, 0.6983958270814684, 0.6983958125114441, 0.6983958098623487, 0.6983958164850871]
Accuracy: 0.51


0.512

In [138]:
compute_accuracy(model_MyMLP, test_loader)

Accuracy: 0.50


0.5